In [20]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, r2_score, roc_auc_score, roc_curve, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [21]:
df=train

In [22]:
df.drop(["Ticket","Cabin","Name","PassengerId"],axis=1,inplace=True)

In [23]:
df=df[~df.Embarked.isnull()]

In [24]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [11]:
df.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
count,889.000000,889.000000,889.000000,712.000000,889.000000,889.000000,889.000000
mean,0.382452,2.311586,0.649044,29.642093,0.524184,0.382452,32.096681
std,0.486260,0.834700,0.477538,14.492933,1.103705,0.806761,49.697504
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,20.000000,0.000000,0.000000,7.895800
50%,0.000000,3.000000,1.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,1.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200


In [25]:
from sklearn.preprocessing import LabelEncoder
df["Sex"]=LabelEncoder().fit_transform(df.Sex)


<ipython-input-25-92a62454e299>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sex"]=LabelEncoder().fit_transform(df.Sex)


In [26]:
df=pd.get_dummies(df,columns=["Pclass","Embarked","Parch","SibSp"],prefix=["Pclass","Embarked","Parch","SibSp"])

In [27]:
from ycimpute.imputer import knnimput
feat_names=list(df)
ndf=np.array(df)
s1=knnimput.KNN(k = 5).complete(ndf)

Imputing row 1/889 with 0 missing, elapsed time: 0.357
Imputing row 101/889 with 1 missing, elapsed time: 0.361
Imputing row 201/889 with 1 missing, elapsed time: 0.361
Imputing row 301/889 with 1 missing, elapsed time: 0.365
Imputing row 401/889 with 0 missing, elapsed time: 0.365
Imputing row 501/889 with 0 missing, elapsed time: 0.369
Imputing row 601/889 with 1 missing, elapsed time: 0.373
Imputing row 701/889 with 0 missing, elapsed time: 0.373
Imputing row 801/889 with 0 missing, elapsed time: 0.377


In [28]:
s1=s1.round()
df=pd.DataFrame(s1,columns=feat_names)
df.isnull().sum()

Survived      0
Sex           0
Age           0
Fare          0
Pclass_1      0
Pclass_2      0
Pclass_3      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
Parch_0       0
Parch_1       0
Parch_2       0
Parch_3       0
Parch_4       0
Parch_5       0
Parch_6       0
SibSp_0       0
SibSp_1       0
SibSp_2       0
SibSp_3       0
SibSp_4       0
SibSp_5       0
SibSp_8       0
dtype: int64

In [29]:
from sklearn.neighbors import LocalOutlierFactor
lof=LocalOutlierFactor()
lof.fit_predict(df)
scores=lof.negative_outlier_factor_
scores

array([-1.46856235, -1.02722794, -1.07930734, -0.98331205, -0.9869583 ,
       -0.99459612, -1.14687565, -0.99045017, -1.24697071, -1.14263669,
       -1.03092231, -1.0886479 , -0.96281536, -1.12011909, -1.21471821,
       -1.1641982 , -0.99543856, -1.08628579, -1.06362046, -1.36568279,
       -0.9882519 , -1.11081549, -1.2196538 , -1.32628095, -0.99608948,
       -1.11035814, -1.02300039, -1.31953441, -0.99738541, -0.99177053,
       -1.0239938 , -1.06577565, -0.99738541, -1.50810659, -0.99205597,
       -1.04214553, -1.2613855 , -0.91981314, -1.10361713, -1.39026035,
       -1.03271924, -1.01636646, -1.014806  , -1.35239282, -1.06082398,
       -0.96932776, -1.06495732, -0.99738541, -1.0954717 , -1.09273901,
       -1.22788252, -0.91981314, -1.02523393, -0.96324092, -1.30061559,
       -1.30804322, -1.38592559, -1.28199167, -0.97226243, -1.11006781,
       -1.40101447, -1.00406439, -0.98305636, -0.97641253, -1.05779561,
       -1.25470274, -1.00468979, -1.2335164 , -1.30227425, -1.28

In [30]:
vector=scores > -1.5

In [31]:
df=df[vector]

In [33]:
x=df.drop("Survived",axis=1)
y=df.Survived

In [34]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20)

models=[LogisticRegression,
       KNeighborsClassifier,
       SVC,
       MLPClassifier,
       DecisionTreeClassifier,
       RandomForestClassifier,
       GradientBoostingClassifier,
       LGBMClassifier,
       XGBClassifier,
       CatBoostClassifier]

import time

def fitter(x_train,x_test,y_train,y_test,model):
        startt=time.time()
        modelf=model().fit(x_train,y_train)
        y_pred=modelf.predict(x_test)
        timer=time.time()-startt
        ascore=accuracy_score(y_test,y_pred)
        print("""
            Model:{}
            accuracy_score:{}
            fit&predict time:{}
            """.format(model.__name__,ascore,timer))
        print("-"*60)


for i in models:
    fitter(x_train,x_test,y_train,y_test,i)

C:\Users\berka\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



            Model:LogisticRegression
            accuracy_score:0.8187134502923976
            fit&predict time:0.08371520042419434
            
------------------------------------------------------------

            Model:KNeighborsClassifier
            accuracy_score:0.695906432748538
            fit&predict time:0.03728318214416504
            
------------------------------------------------------------

            Model:SVC
            accuracy_score:0.6432748538011696
            fit&predict time:0.05859875679016113
            
------------------------------------------------------------


C:\Users\berka\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(



            Model:MLPClassifier
            accuracy_score:0.8362573099415205
            fit&predict time:2.284754753112793
            
------------------------------------------------------------

            Model:DecisionTreeClassifier
            accuracy_score:0.7719298245614035
            fit&predict time:0.025532245635986328
            
------------------------------------------------------------

            Model:RandomForestClassifier
            accuracy_score:0.7953216374269005
            fit&predict time:0.7312619686126709
            
------------------------------------------------------------

            Model:GradientBoostingClassifier
            accuracy_score:0.8538011695906432
            fit&predict time:0.26325178146362305
            
------------------------------------------------------------

            Model:LGBMClassifier
            accuracy_score:0.8304093567251462
            fit&predict time:0.17153000831604004
            
---------------------

C:\Users\berka\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

152:	learn: 0.4027115	total: 540ms	remaining: 2.99s
153:	learn: 0.4022544	total: 543ms	remaining: 2.98s
154:	learn: 0.4016381	total: 547ms	remaining: 2.98s
155:	learn: 0.4013849	total: 549ms	remaining: 2.97s
156:	learn: 0.4007431	total: 553ms	remaining: 2.97s
157:	learn: 0.4000330	total: 556ms	remaining: 2.96s
158:	learn: 0.3997696	total: 560ms	remaining: 2.96s
159:	learn: 0.3992897	total: 563ms	remaining: 2.96s
160:	learn: 0.3986574	total: 567ms	remaining: 2.95s
161:	learn: 0.3978070	total: 570ms	remaining: 2.95s
162:	learn: 0.3971684	total: 574ms	remaining: 2.95s
163:	learn: 0.3967554	total: 578ms	remaining: 2.94s
164:	learn: 0.3963254	total: 581ms	remaining: 2.94s
165:	learn: 0.3957379	total: 585ms	remaining: 2.94s
166:	learn: 0.3953233	total: 589ms	remaining: 2.94s
167:	learn: 0.3945361	total: 593ms	remaining: 2.94s
168:	learn: 0.3939760	total: 596ms	remaining: 2.93s
169:	learn: 0.3936075	total: 600ms	remaining: 2.93s
170:	learn: 0.3931589	total: 603ms	remaining: 2.92s
171:	learn: 

351:	learn: 0.3459699	total: 1.34s	remaining: 2.47s
352:	learn: 0.3458815	total: 1.34s	remaining: 2.46s
353:	learn: 0.3455194	total: 1.35s	remaining: 2.46s
354:	learn: 0.3454370	total: 1.35s	remaining: 2.46s
355:	learn: 0.3451872	total: 1.36s	remaining: 2.45s
356:	learn: 0.3448830	total: 1.36s	remaining: 2.45s
357:	learn: 0.3448622	total: 1.36s	remaining: 2.44s
358:	learn: 0.3445393	total: 1.37s	remaining: 2.44s
359:	learn: 0.3442893	total: 1.37s	remaining: 2.44s
360:	learn: 0.3440473	total: 1.37s	remaining: 2.43s
361:	learn: 0.3438515	total: 1.38s	remaining: 2.43s
362:	learn: 0.3435865	total: 1.38s	remaining: 2.42s
363:	learn: 0.3433822	total: 1.38s	remaining: 2.42s
364:	learn: 0.3431389	total: 1.39s	remaining: 2.41s
365:	learn: 0.3428139	total: 1.39s	remaining: 2.41s
366:	learn: 0.3425988	total: 1.39s	remaining: 2.4s
367:	learn: 0.3425342	total: 1.4s	remaining: 2.4s
368:	learn: 0.3424847	total: 1.4s	remaining: 2.4s
369:	learn: 0.3422762	total: 1.4s	remaining: 2.39s
370:	learn: 0.3420

550:	learn: 0.3179223	total: 1.98s	remaining: 1.61s
551:	learn: 0.3178645	total: 1.98s	remaining: 1.61s
552:	learn: 0.3177387	total: 1.99s	remaining: 1.61s
553:	learn: 0.3177164	total: 1.99s	remaining: 1.6s
554:	learn: 0.3176364	total: 1.99s	remaining: 1.6s
555:	learn: 0.3175066	total: 2s	remaining: 1.59s
556:	learn: 0.3174267	total: 2s	remaining: 1.59s
557:	learn: 0.3172082	total: 2s	remaining: 1.58s
558:	learn: 0.3171331	total: 2s	remaining: 1.58s
559:	learn: 0.3168444	total: 2.01s	remaining: 1.58s
560:	learn: 0.3166987	total: 2.01s	remaining: 1.57s
561:	learn: 0.3163471	total: 2.01s	remaining: 1.57s
562:	learn: 0.3161790	total: 2.01s	remaining: 1.56s
563:	learn: 0.3161005	total: 2.02s	remaining: 1.56s
564:	learn: 0.3159546	total: 2.02s	remaining: 1.56s
565:	learn: 0.3157820	total: 2.02s	remaining: 1.55s
566:	learn: 0.3156976	total: 2.03s	remaining: 1.55s
567:	learn: 0.3156100	total: 2.03s	remaining: 1.54s
568:	learn: 0.3154878	total: 2.04s	remaining: 1.54s
569:	learn: 0.3153968	tota

753:	learn: 0.2919697	total: 2.62s	remaining: 856ms
754:	learn: 0.2918526	total: 2.63s	remaining: 853ms
755:	learn: 0.2917711	total: 2.63s	remaining: 849ms
756:	learn: 0.2915225	total: 2.63s	remaining: 846ms
757:	learn: 0.2914674	total: 2.64s	remaining: 842ms
758:	learn: 0.2913017	total: 2.64s	remaining: 838ms
759:	learn: 0.2911028	total: 2.64s	remaining: 835ms
760:	learn: 0.2910106	total: 2.65s	remaining: 831ms
761:	learn: 0.2908769	total: 2.65s	remaining: 828ms
762:	learn: 0.2907365	total: 2.65s	remaining: 824ms
763:	learn: 0.2905681	total: 2.66s	remaining: 821ms
764:	learn: 0.2903359	total: 2.66s	remaining: 817ms
765:	learn: 0.2902343	total: 2.66s	remaining: 813ms
766:	learn: 0.2900224	total: 2.67s	remaining: 810ms
767:	learn: 0.2898619	total: 2.67s	remaining: 806ms
768:	learn: 0.2898224	total: 2.67s	remaining: 803ms
769:	learn: 0.2896751	total: 2.67s	remaining: 799ms
770:	learn: 0.2895653	total: 2.68s	remaining: 796ms
771:	learn: 0.2894399	total: 2.68s	remaining: 792ms
772:	learn: 

928:	learn: 0.2699677	total: 3.27s	remaining: 250ms
929:	learn: 0.2698498	total: 3.27s	remaining: 246ms
930:	learn: 0.2695894	total: 3.28s	remaining: 243ms
931:	learn: 0.2695052	total: 3.28s	remaining: 239ms
932:	learn: 0.2694186	total: 3.29s	remaining: 236ms
933:	learn: 0.2692821	total: 3.29s	remaining: 232ms
934:	learn: 0.2691929	total: 3.29s	remaining: 229ms
935:	learn: 0.2689461	total: 3.3s	remaining: 226ms
936:	learn: 0.2688330	total: 3.3s	remaining: 222ms
937:	learn: 0.2687787	total: 3.31s	remaining: 219ms
938:	learn: 0.2687204	total: 3.31s	remaining: 215ms
939:	learn: 0.2686965	total: 3.32s	remaining: 212ms
940:	learn: 0.2684611	total: 3.32s	remaining: 208ms
941:	learn: 0.2683176	total: 3.32s	remaining: 205ms
942:	learn: 0.2681400	total: 3.33s	remaining: 201ms
943:	learn: 0.2680967	total: 3.33s	remaining: 198ms
944:	learn: 0.2680493	total: 3.33s	remaining: 194ms
945:	learn: 0.2680004	total: 3.34s	remaining: 191ms
946:	learn: 0.2678115	total: 3.35s	remaining: 187ms
947:	learn: 0.

In [35]:
### I will use GBM and LGBM

In [ ]:
#GBM default params:
#learning_rate=0.1
#n_estimators=100
#min_samples_split=2
#min_samples_leaf=1
#max_depth=3


gbm_params={
    "learning_rate":[0.01,0.1,0.3],
    "n_estimators":[100,500,2000],
    "min_samples_split":[2,4,6,7],
    "min_samples_leaf":[1,2,3],
    "max_depth":[3,5,7]
}

gbm_cv_model=GridSearchCV(GradientBoostingClassifier(),gbm_params,cv=5,verbose=2,n_jobs=-1).fit(x_train,y_train)

In [42]:
gbm_cv_model.best_params_

{'learning_rate': 0.01,
 'max_depth': 5,
 'min_samples_leaf': 2,
 'min_samples_split': 4,
 'n_estimators': 500}

In [36]:
gbm_tuned=GradientBoostingClassifier(learning_rate=0.01,
                                    max_depth=5,
                                    min_samples_leaf=2,
                                    min_samples_split=4,
                                    n_estimators=500).fit(x_train,y_train)

In [45]:
y_pred=gbm_tuned.predict(x_test)
accuracy_score(y_test,y_pred)

0.8538011695906432

In [46]:
## same with default model, lol

In [ ]:
lgbm_params={
    "learning_rate":[0.01,0.1,0.3],
    "num_leaves":[25,31,40,50],
    "n_estimators":[50,100,500,2000],
    "min_child_samples":[10,20,30],
    "num_iterations":[100,500,1000],
    "max_bin":[255,305,400]
}

lgbm_cv_model=GridSearchCV(LGBMClassifier(),lgbm_params,cv=5,verbose=2,n_jobs=-1).fit(x_train,y_train)


In [48]:
lgbm_cv_model.best_params_

{'learning_rate': 0.01,
 'max_bin': 255,
 'min_child_samples': 10,
 'n_estimators': 50,
 'num_iterations': 500,
 'num_leaves': 31}

In [51]:
lgbm_tuned=LGBMClassifier(learning_rate=0.01,
                         max_bin=255,
                         min_child_samples=10,
                         n_estimators=1000,
                         num_iterations=500,
                         num_leaves=31,
                         ).fit(x_train,y_train,
                                 eval_set=[(x_test,y_test)],
                                 eval_metric="error")

[1]	valid_0's binary_error: 0.368421	valid_0's binary_logloss: 0.654652
[2]	valid_0's binary_error: 0.368421	valid_0's binary_logloss: 0.649513
[3]	valid_0's binary_error: 0.368421	valid_0's binary_logloss: 0.644493
[4]	valid_0's binary_error: 0.368421	valid_0's binary_logloss: 0.639851
[5]	valid_0's binary_error: 0.368421	valid_0's binary_logloss: 0.635052
[6]	valid_0's binary_error: 0.368421	valid_0's binary_logloss: 0.63036
[7]	valid_0's binary_error: 0.368421	valid_0's binary_logloss: 0.625721
[8]	valid_0's binary_error: 0.368421	valid_0's binary_logloss: 0.621235
[9]	valid_0's binary_error: 0.368421	valid_0's binary_logloss: 0.617095
[10]	valid_0's binary_error: 0.368421	valid_0's binary_logloss: 0.612798
[11]	valid_0's binary_error: 0.368421	valid_0's binary_logloss: 0.608541
[12]	valid_0's binary_error: 0.368421	valid_0's binary_logloss: 0.604431
[13]	valid_0's binary_error: 0.368421	valid_0's binary_logloss: 0.600399
[14]	valid_0's binary_error: 0.368421	valid_0's binary_loglos

[185]	valid_0's binary_error: 0.157895	valid_0's binary_logloss: 0.400709
[186]	valid_0's binary_error: 0.157895	valid_0's binary_logloss: 0.400736
[187]	valid_0's binary_error: 0.157895	valid_0's binary_logloss: 0.400689
[188]	valid_0's binary_error: 0.157895	valid_0's binary_logloss: 0.400763
[189]	valid_0's binary_error: 0.157895	valid_0's binary_logloss: 0.400714
[190]	valid_0's binary_error: 0.157895	valid_0's binary_logloss: 0.400642
[191]	valid_0's binary_error: 0.157895	valid_0's binary_logloss: 0.400557
[192]	valid_0's binary_error: 0.157895	valid_0's binary_logloss: 0.40036
[193]	valid_0's binary_error: 0.157895	valid_0's binary_logloss: 0.400271
[194]	valid_0's binary_error: 0.157895	valid_0's binary_logloss: 0.400128
[195]	valid_0's binary_error: 0.157895	valid_0's binary_logloss: 0.400067
[196]	valid_0's binary_error: 0.157895	valid_0's binary_logloss: 0.39986
[197]	valid_0's binary_error: 0.157895	valid_0's binary_logloss: 0.399665
[198]	valid_0's binary_error: 0.157895	v

[366]	valid_0's binary_error: 0.163743	valid_0's binary_logloss: 0.413494
[367]	valid_0's binary_error: 0.163743	valid_0's binary_logloss: 0.413773
[368]	valid_0's binary_error: 0.163743	valid_0's binary_logloss: 0.413674
[369]	valid_0's binary_error: 0.163743	valid_0's binary_logloss: 0.413607
[370]	valid_0's binary_error: 0.169591	valid_0's binary_logloss: 0.413815
[371]	valid_0's binary_error: 0.169591	valid_0's binary_logloss: 0.413792
[372]	valid_0's binary_error: 0.169591	valid_0's binary_logloss: 0.414121
[373]	valid_0's binary_error: 0.169591	valid_0's binary_logloss: 0.414393
[374]	valid_0's binary_error: 0.169591	valid_0's binary_logloss: 0.414341
[375]	valid_0's binary_error: 0.169591	valid_0's binary_logloss: 0.414385
[376]	valid_0's binary_error: 0.169591	valid_0's binary_logloss: 0.41445
[377]	valid_0's binary_error: 0.169591	valid_0's binary_logloss: 0.414535
[378]	valid_0's binary_error: 0.169591	valid_0's binary_logloss: 0.414575
[379]	valid_0's binary_error: 0.169591	

In [59]:
lgbm_tuned=LGBMClassifier(learning_rate=0.01,
                         max_bin=255,
                         min_child_samples=10,
                         n_estimators=100,
                         num_iterations=500,
                         num_leaves=31,
                         ).fit(x_train,y_train,)
y_pred=lgbm_tuned.predict(x_test)
accuracy_score(y_test,y_pred)

0.8362573099415205

In [63]:
## minimal improvement of accuracy score (0.006),
## I will use gbm_tuned

In [37]:
df=pd.read_csv("test.csv")
df.drop(["Ticket","Cabin","Name","PassengerId"],axis=1,inplace=True)
df["Sex"]=LabelEncoder().fit_transform(df.Sex)
df=pd.get_dummies(df,columns=["Pclass","Embarked","Parch","SibSp"],prefix=["Pclass","Embarked","Parch","SibSp"])
df.columns

Index(['Sex', 'Age', 'Fare', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Embarked_C',
       'Embarked_Q', 'Embarked_S', 'Parch_0', 'Parch_1', 'Parch_2', 'Parch_3',
       'Parch_4', 'Parch_5', 'Parch_6', 'Parch_9', 'SibSp_0', 'SibSp_1',
       'SibSp_2', 'SibSp_3', 'SibSp_4', 'SibSp_5', 'SibSp_8'],
      dtype='object')

In [38]:
df=df.drop("Parch_9",axis=1)

In [39]:
from ycimpute.imputer import knnimput
feat_names=list(df)
ndf=np.array(df)
s1=knnimput.KNN(k = 5).complete(ndf)
s1=s1.round()
df=pd.DataFrame(s1,columns=feat_names)
df.isnull().sum()

Imputing row 1/418 with 0 missing, elapsed time: 0.076
Imputing row 101/418 with 0 missing, elapsed time: 0.078
Imputing row 201/418 with 1 missing, elapsed time: 0.078
Imputing row 301/418 with 0 missing, elapsed time: 0.082
Imputing row 401/418 with 0 missing, elapsed time: 0.082


Sex           0
Age           0
Fare          0
Pclass_1      0
Pclass_2      0
Pclass_3      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
Parch_0       0
Parch_1       0
Parch_2       0
Parch_3       0
Parch_4       0
Parch_5       0
Parch_6       0
SibSp_0       0
SibSp_1       0
SibSp_2       0
SibSp_3       0
SibSp_4       0
SibSp_5       0
SibSp_8       0
dtype: int64

In [40]:
test=df

In [41]:
preds=gbm_tuned.predict(test)
preds

array([0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
       1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1.,
       0., 1., 0., 1., 1.

In [42]:
results=pd.DataFrame(preds,columns=["Survived"],index=range(892,1310))

In [43]:
results.index.name="PassengerId"

In [44]:
results

,Survived
PassengerId,
892,0.0
893,0.0
894,0.0
895,0.0
896,1.0
...,...
1305,0.0
1306,1.0
1307,0.0


In [46]:
res=results.astype("int64")

In [48]:
res.to_csv("predv2.csv")